##Construcción de un Pipeline completo con MLlib para clasificación

###Contexto:
MLlib permite automatizar flujos de Machine Learning con pipelines que combinan transformaciones, entrenamiento y evaluación. Esta actividad entrena a los estudiantes en su uso completo con datos distribuidos.

###Consigna:
A partir de un dataset con variables categóricas y numéricas, construye un pipeline que incluya indexación, ensamblado de features, entrenamiento con regresión logística y evaluación con AUC o F1-score.

###Paso a paso:

1- Lee un CSV distribuido usando spark.read.csv() y explora su estructura.

2- Aplica StringIndexer para convertir variables categóricas.

3- Usa VectorAssembler para combinar las columnas de entrada.

4- Define el modelo de clasificación (LogisticRegression).

5- Encadena las etapas en un Pipeline.

6- Divide en train y test, y entrena el modelo con .fit().

7- Evalúe con BinaryClassificationEvaluator.

8- Muestra resultados e interpreta las métricas.


In [2]:
# Instalar PySpark si es necesario
!pip install pyspark

# Importar librerías
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from google.colab import files

# Subir CSV desde tu computadora
print("Selecciona tu archivo CSV:")
uploaded = files.upload()

# Obtener el nombre del archivo subido
csv_file = list(uploaded.keys())[0]
print(f"Archivo subido: {csv_file}")

#Crear SparkSession
spark = SparkSession.builder \
    .appName("PipelineMLlibClasificacion") \
    .getOrCreate()

# Leer CSV subido
df = spark.read.csv(csv_file, header=True, inferSchema=True)

# Explorar dataset
df.show(5)
df.printSchema()

# dentificar columnas categóricas y numéricas
categorical_cols = ['categoria1', 'categoria2']
numeric_cols = [col for col in df.columns if col not in categorical_cols + ['label']]

#Indexar variables categóricas
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid='keep')
            for col in categorical_cols]

#Combinar features
feature_cols = [c + "_index" for c in categorical_cols] + numeric_cols
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Definir modelo de clasificación
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Crear pipeline
stages = indexers + [assembler, lr]
pipeline = Pipeline(stages=stages)

# Dividir dataset en train y test
train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

# Entrenar modelo
model = pipeline.fit(train_df)

#Predecir sobre test
predictions = model.transform(test_df)
predictions.select("label", "prediction", "probability").show(5)

#Evaluar modelo
evaluator_auc = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
evaluator_f1 = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderPR")  # Aproximación a F1

auc = evaluator_auc.evaluate(predictions)
f1_score = evaluator_f1.evaluate(predictions)

print(f"\n✅ Métricas del modelo:")
print(f"AUC: {auc:.4f}")
print(f"F1-score (aprox.): {f1_score:.4f}")



Selecciona tu archivo CSV:


Saving dataset_categorico.csv to dataset_categorico.csv
Archivo subido: dataset_categorico.csv
+----------+----------+----+----+-----+
|categoria1|categoria2|num1|num2|label|
+----------+----------+----+----+-----+
|         A|         X|  10| 0.5|    1|
|         B|         Y|  15| 0.7|    0|
|         A|         X|  10| 0.2|    1|
|         C|         Z|  20| 0.9|    0|
|         B|         Y|  15| 0.4|    0|
+----------+----------+----+----+-----+
only showing top 5 rows

root
 |-- categoria1: string (nullable = true)
 |-- categoria2: string (nullable = true)
 |-- num1: integer (nullable = true)
 |-- num2: double (nullable = true)
 |-- label: integer (nullable = true)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|    1|       1.0|[0.18232659901064...|
|    0|       1.0|[0.29214727121469...|
|    0|       0.0|[0.54741266313554...|
|    1|       0.0|[0.54741266313554...|
+-----+----------+--------------------+

##Interpretación general

El modelo está funcionando, pero los datos son muy pocos para entrenar un modelo robusto.

Algunas predicciones incorrectas indican que el modelo no tiene suficiente información para generalizar.

La combinación de variables categóricas y numéricas fue procesada correctamente gracias a StringIndexer + VectorAssembler.

La métrica F1 alta indica que cuando predice positivo lo hace con cierto acierto, pero el AUC moderado muestra que aún confunde clases.

##Recomendaciones

Aumentar la cantidad de datos para entrenar mejor el modelo.

Evaluar otras técnicas de clasificación (Random Forest, Gradient Boosting) que capturen relaciones no lineales.

Revisar si las categorías tienen suficiente representación para aprender.